<a href="https://colab.research.google.com/github/psriraj17/sentiment_analysis/blob/main/sentiment_analysis_NB_nltk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
#The following given below are the predefined libraries of python. We have imported various libraries like Pandas, Numpy and a lot of other algorithm libraries too. 
#These libraries are used iteratively to reduce the time required to code.
import pandas as pd
import io
import numpy as np
import re
import pickle
import nltk
import string
from nltk.tokenize import RegexpTokenizer
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import sklearn
from sklearn import naive_bayes
!pip install emot
import re
import pickle
from emot.emo_unicode import UNICODE_EMOJI
from emot.emo_unicode import EMOTICONS_EMO

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
#mounting google drive
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/'MyDrive'
%ls

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive
'10 22 2021.gdoc'
'An Intelligent System for Waste Materials Segregation Using IoT and Deep Learning.pdf'
 baby.jpeg
 bigdata_project.gslides
 breast-cancer.data
'car (1).data'
 checkmodel.h5
 CN2_screenshot.gdoc
 CN_Homework1.gdoc
'CN_Homework 2.gdoc'
 CN_homework3.gdoc
 CN_pipeline.gdoc
'Colab Notebooks'/
'COMPUTER VISION ASSIGNMENT 3.gdoc'
'Copy of Pepsi and Vending Refund Request.gform'@
'Copy of Pepsi and Vending Refund Request OLD.gform'
'Copy of Pepsi and Vending Refunds (Responses) OLD.gsheet'
'Copy of Student Development Bank Deposit Form.gform'@
'CSO Reimbursement Request Form - Pending.gsheet'
'CV - 0121.gdoc'
 CV_FinalProject.gdoc
'Data Science Notes.gdoc'
'Deep Learning Notebooks'/
 DL_Project_GI_Tracking.gslides
 ecoli.data
'Emma Project.gdoc'
'Emma Research Project.gdoc'
'E(mpower) Funds Request Form (File responses)'/
'E

In [4]:
#Loading and reading the dataset
import csv
train_df = pd.read_csv('train-v2.tsv', sep="\t", header=None, quoting = csv.QUOTE_NONE)
train_df.head()

,0,1
0,0,@USER @USER a sicrhau bod mwy o arian poced 'd...
1,1,Parti Dolig da gyda tim swyddfa canolog @USER ...
2,0,@USER yeaah ma fe yn wir. ( oh well.
3,1,@USER hahaha idk. 3am oedd y bws ti?
4,0,@USER dwim yn gal llun ohoni?


In [5]:
#Renaming the column names to Target and Text to ensure better readability.
train_df.rename(columns={0: 'Target', 1: 'Text'}, inplace=True)
train_df

,Target,Text
0,0,@USER @USER a sicrhau bod mwy o arian poced 'd...
1,1,Parti Dolig da gyda tim swyddfa canolog @USER ...
2,0,@USER yeaah ma fe yn wir. ( oh well.
3,1,@USER hahaha idk. 3am oedd y bws ti?
4,0,@USER dwim yn gal llun ohoni?
...,...,...
79995,1,"@USER @USER diolch yn fawr am ein diweddaru, r..."
79996,1,@USER @USER @USER diolch! Digon o opsiynne felly!
79997,0,mor hungover heddiw 🔫
79998,1,"@USER eidiaaaal, awni dydd llun nesa ia? Y air..."


In [6]:
data=train_df[['Text','Target']]
data['Text']

0        @USER @USER a sicrhau bod mwy o arian poced 'd...
1        Parti Dolig da gyda tim swyddfa canolog @USER ...
2                     @USER yeaah ma fe yn wir. ( oh well.
3                     @USER hahaha idk. 3am oedd y bws ti?
4                            @USER dwim yn gal llun ohoni?
                               ...                        
79995    @USER @USER diolch yn fawr am ein diweddaru, r...
79996    @USER @USER @USER diolch! Digon o opsiynne felly!
79997                                mor hungover heddiw 🔫
79998    @USER eidiaaaal, awni dydd llun nesa ia? Y air...
79999    @USER ahaha, aino maisho mynadd! Erm o be dwin...
Name: Text, Length: 80000, dtype: object

In [7]:
data_pos = data[data['Target'] == 1]
data_neg = data[data['Target'] == 0]

In [8]:
df = pd.concat([data_pos, data_neg])
df.head()

,Text,Target
1,Parti Dolig da gyda tim swyddfa canolog @USER ...,1
3,@USER hahaha idk. 3am oedd y bws ti?,1
6,@USER Diolch! Dweud wrtho Manon fi wedi txto h...,1
8,"@USER mor browd, fe fyddai yn dangos ti faint ...",1
10,@USER ond paid a meiddio darllen y tweet yma a...,1


In [9]:
df['Text']=df['Text'].str.lower()
df['Text'].tail()

79987    @user fina fyd! ti'n sgol fory? sgenai'm mynad...
79988    @user all white wash, a playsuit gwyn fo tiny ...
79992                         @user cymryd un fi lawr fory
79997                                mor hungover heddiw 🔫
79999    @user ahaha, aino maisho mynadd! erm o be dwin...
Name: Text, dtype: object

In [10]:
#This is to remove the @user string. This piece of code is taken from https://www.analyticsvidhya.com/blog/2021/06/twitter-sentiment-analysis-a-nlp-use-case-for-beginners/
pattern = ['@USER','@user','user']
def cleaning_pattern(text):
    return " ".join([word for word in str(text).split() if word not in pattern])
df['Text'] = df['Text'].apply(lambda text: cleaning_pattern(text))
df['Text'].head()

1     parti dolig da gyda tim swyddfa canolog ty gwy...
3                        hahaha idk. 3am oedd y bws ti?
6          diolch! dweud wrtho manon fi wedi txto hi! x
8     mor browd, fe fyddai yn dangos ti faint mor aw...
10    ond paid a meiddio darllen y tweet yma allan a...
Name: Text, dtype: object

In [11]:
#to remove punctuation
import string
english_punctuations = string.punctuation
punctuations_list = english_punctuations
def cleaning_punctuations(text):
    translator = str.maketrans('', '', punctuations_list)
    return text.translate(translator)
df['Text']= df['Text'].apply(lambda x: cleaning_punctuations(x))
df['Text'].tail()

79987     fina fyd tin sgol fory sgenaim mynadd o gwbwl de
79988    all white wash a playsuit gwyn fo tiny bit o p...
79992                               cymryd un fi lawr fory
79997                                mor hungover heddiw 🔫
79999    ahaha aino maisho mynadd erm o be dwin wbod ia...
Name: Text, dtype: object

In [30]:
#remove redudant words
def cleaning_redundant_words(text):
    return re.sub(r'(.)1+', r'1', text)
df['Text'] = df['Text'].apply(lambda x: cleaning_redundant_words(x))
df['Text'].tail()

79987     fina fyd tin sgol fory sgenaim mynadd o gwbwl de
79988    all white wash a playsuit gwyn fo tiny bit o p...
79992                               cymryd un fi lawr fory
79997                                 mor hungover heddiw 
79999    ahaha aino maisho mynadd erm o be dwin wbod ia...
Name: Text, dtype: object

In [13]:
#remove numbers
def cleaning_numerical_data(data):
    return re.sub('[0-9]+', '', data)
df['Text'] = df['Text'].apply(lambda x: cleaning_numerical_data(x))
df['Text'].tail()

79987     fina fyd tin sgol fory sgenaim mynadd o gwbwl de
79988    all white wash a playsuit gwyn fo tiny bit o p...
79992                               cymryd un fi lawr fory
79997                                mor hungover heddiw 🔫
79999    ahaha aino maisho mynadd erm o be dwin wbod ia...
Name: Text, dtype: object

In [37]:
#delete the emoticons. we used the emot library to remove them. This piece of code was taken from https://medium.com/geekculture/text-preprocessing-how-to-handle-emoji-emoticon-641bbfa6e9e7 
#it was modified according to our use case only.
import re
def delete_emoji(string):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F" 
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

In [38]:
df['Text']=df['Text'].apply(delete_emoji)

In [39]:
delete_emoji(df['Text'][79997])

'mor hungover heddiw '

In [18]:
X=df.Text
y=df.Target
X

1        parti dolig da gyda tim swyddfa canolog ty gwy...
3                              hahaha idk am oedd y bws ti
6               diolch dweud wrtho manon fi wedi txto hi x
8        mor browd fe fyddai yn dangos ti faint mor awe...
10       ond paid a meiddio darllen y tweet yma allan a...
                               ...                        
79987     fina fyd tin sgol fory sgenaim mynadd o gwbwl de
79988    all white wash a playsuit gwyn fo tiny bit o p...
79992                               cymryd un fi lawr fory
79997                                 mor hungover heddiw 
79999    ahaha aino maisho mynadd erm o be dwin wbod ia...
Name: Text, Length: 80000, dtype: object

In [19]:
#Splitting the dataset into training, testing and validation dataset
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.20, random_state =42)

In [20]:
#to convert into sparse matrix format for input
vectoriser = TfidfVectorizer(ngram_range=(1,2), max_features=500000)
vectoriser.fit(X_train)
print('No. of feature_words: ', len(vectoriser.get_feature_names()))

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


No. of feature_words:  447693


In [24]:
#vectorising the training and testing data
X_train = vectoriser.transform(X_train)
X_test  = vectoriser.transform(X_test)

In [25]:
#model training
from sklearn.naive_bayes import BernoulliNB
BNBmodel = BernoulliNB()
BNBmodel.fit(X_train, y_train)
y_pred1 = BNBmodel.predict(X_test)

In [26]:
#printing the accuracy
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, y_pred1))

0.7411875


In [35]:
def predict_anything_goes(tweet):
  # do something complicated here
  tweet = cleaning_pattern(tweet)
  tweet = cleaning_punctuations(tweet)
  tweet = cleaning_redundant_words(tweet)
  tweet = cleaning_numerical_data(tweet)
  tweet = vectoriser.transform([tweet])
  y_pred = BNBmodel.predict(tweet)
  return y_pred.any()

In [36]:
def evaluate():
  total = 0
  correct_from_scratch = 0
  correct_anything_goes = 0
  testfile = open('test.tsv', 'r')
  for line in testfile:
    total += 1
    pieces = line.rstrip("\n").split("\t")
    #if predict_from_scratch(pieces[1]) == int(pieces[0]):
     # correct_from_scratch += 1
    if predict_anything_goes(pieces[1]) == int(pieces[0]):
      correct_anything_goes += 1
  return (correct_from_scratch/total, correct_anything_goes/total)

In [29]:
#test set prediction
evaluate()

(0.0, 0.8254)